<a href="https://colab.research.google.com/github/Monishgalla/Classifying-different-land-uses-in-satellite-images-using-CNN/blob/main/Final2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Set the path to your dataset in .tif format
data_dir = '/content/drive/MyDrive/dataset/UCMerced_LandUse/tif images'

# Set the number of classes and batch size
num_classes = 21
batch_size = 32

# Load the data from the dataset directory
data = []
labels = []

class_names = sorted(os.listdir(data_dir))
class_to_label = {class_name: label for label, class_name in enumerate(class_names)}

for class_name in class_names:
    class_dir = os.path.join(data_dir, class_name)
    if os.path.isdir(class_dir):
        for image_name in os.listdir(class_dir):
            if image_name.endswith('.tif'):
                image_path = os.path.join(class_dir, image_name)
                image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
                image = tf.keras.preprocessing.image.img_to_array(image)
                data.append(image)
                labels.append(class_to_label[class_name])

# Convert the data and labels to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Convert labels to one-hot encoded format
labels = to_categorical(labels, num_classes=num_classes)

# Perform train-test split
train_data, val_data, train_labels, val_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(train_data, train_labels, batch_size=batch_size)
val_generator = val_datagen.flow(val_data, val_labels, batch_size=batch_size)

# Load the VGG16 model without the top classification layer
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in vgg_model.layers:
    layer.trainable = False

# Create a new model
model = Sequential()
model.add(vgg_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Training the model
epochs = 5

model.fit(
    train_generator,
    steps_per_epoch=len(train_data) // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=len(val_data) // batch_size,
    callbacks=[early_stopping]
)

# Save the trained model
model.save('/content/drive/MyDrive/trained_model.h5')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/5
52/52 [==============================] - 855s 17s/step - loss: 2.0744 - accuracy: 0.5000 - val_loss: 0.8222 - val_accuracy: 0.7620
Epoch 2/5
52/52 [==============================] - 809s 16s/step - loss: 0.6953 - accuracy: 0.7834 - val_loss: 0.6131 - val_accuracy: 0.7957
Epoch 3/5
52/52 [==============================] - 852s 16s/step - loss: 0.5985 - accuracy: 0.8051 - val_loss: 0.5811 - val_accuracy: 0.8125
Epoch 4/5
52/52 [==============================] - 806s 16s/step - loss: 0.4157 - accuracy: 0.8658 - val_loss: 0.4587 - val_accuracy: 0.8702
Epoch 5/5
52/52 [==============================] - 803s 16s/step - loss: 0.3353 - accuracy: 0.8893 - val_loss: 0.4124 - val_accuracy: 0.8726


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
